#### Student Name:
#### Student ID:

## Speech Formants with Linear Predictive Coding, Vocoder (Mister Blue Sky)

Instructions: 

* This notebook is an interactive assignment; please read and follow the instructions in each cell. 

* Cells that require your input (in the form of code or written response) will have 'Question #' above.

* After completing the assignment, please submit this notebook and its pdf printout and all sound files. 

## Speech Formants and LPC

In this section, you will synthesize vowel sounds, and investigate the frequencies in vowels from your own voice. 

In [ ]:
import numpy as np
import pandas as pd
from collections import Counter
from numpy.random import multinomial as randm
from numpy import where
import scipy.signal as si
import matplotlib.pyplot as plt
from matplotlib import patches
import IPython.display as ipd
import librosa
import scipy
import librosa.display as ld
import music21
from music21 import midi as midi21
from music21 import stream
from jchord.progressions import ChordProgression, MidiConversionSettings
from scipy.io import wavfile as wavfile 
import copy


Fdict = {
    'mystery_1':[[328, 2208, 2885],[27,80,575]],
    'mystery_2':[[504, 868, 2654],[62,   108,  299]],
    'mystery_3':[[700, 1220, 2600],[130,   70,  160]]
    } # Formant frequencies in Hz

def excitation(f0,jitt,dur,nharm=None,unvoiced=False):
    w0T = 2*np.pi*f0/fs

    if nharm == None:
        nharm = int((fs/2)/f0) # number of harmonics
    nsamps = fs*dur
    sig = np.zeros(nsamps)
    ph = np.random.uniform(size=nsamps)*2*np.pi
    n = np.arange(nsamps)

    if unvoiced:
        sig = np.random.normal(size=nsamps)
    else:
    # Synthesize bandlimited impulse train
        for i in range(1,nharm):
            sig = sig + np.cos(i*w0T*n + jitt*ph)
    
    sig = sig/max(sig)
    return sig

def voca(sig,F,Fb):
    R = np.exp(-np.pi*Fb/fs);     # Pole radii
    theta = 2*np.pi*F/fs;     # Pole angles
    poles = R * np.exp(1j*theta) # Poles[B,A] = zpk2tf(0,np.concatenate((poles, np.conj(poles))),1) # Filter from zeros and poles
    
    [B,A] = si.zpk2tf(0,np.concatenate((poles, np.conj(poles))),1) # Filter from zeros and poles

    speech = si.lfilter(B, A, sig)
    speech = speech/np.std(speech)
    return speech,B,A

fs = 8192 # 22050  % Sampling rate in Hz ("telephone quality" for speed)

vowels = list(Fdict.keys())
f0 = 150 # Pitch in Hz
dur = 1 #one second in duration
ji = 0.1 #0.1
ex = excitation(f0,ji,dur)

text = ['mystery_1','mystery_2','mystery_3']

speech = np.zeros(1)
for t in text:
    F = np.array(Fdict[t][0])
    Fb = np.array(Fdict[t][1])
    print(t)

    vow,B,A = voca(ex,F,Fb)
    speech = np.concatenate((speech,vow))

speech = speech/np.std(speech)
plt.plot(speech)

In [ ]:
ipd.Audio(speech, rate=fs) 

##### Question 1 (10 points)

Based on the audio output, what vowels were synthesized as mystery_1, mystery_2, and mystery_3? 
Please specify using a word; for example, if you heard an 'oo' sound as in 'hoot', you may answer with the word "hoot". 

mystery_1:

mystery_2:

mystery_3:

Now we will examine just one vowel in greater detail. 
Select one mystery vowel to analyze below: 

In [ ]:
%matplotlib inline  

### Modify the line below:
text = ['SELECT_ONE_MYSTERY_VOWEL']

speech = np.zeros(1)
for t in text:
    F = np.array(Fdict[t][0])
    Fb = np.array(Fdict[t][1])
    print(t)

    vow,B,A = voca(ex,F,Fb)
    speech = np.concatenate((speech,vow))

speech = speech/np.std(speech)
plt.plot(speech)

In [5]:
# Plot the power spectral density (PSD)
plt.psd(speech, 1024)
plt.show()

In [ ]:
lpc_order = 10
s = speech

a = librosa.core.lpc(s, order=lpc_order)
print(a)
s_hat = scipy.signal.lfilter([0] + -1*a[1:], [1], s)
s_err = s[1:] - s_hat[:-1]
plt.plot(s[1:])
plt.plot(s_hat[:-1], linestyle='--')
plt.legend(['y', 'y_hat'])
plt.title('LP Model Forward Prediction')
plt.show()

In [ ]:
plt.plot(s[1:101])
plt.plot(s_hat[:100])
plt.legend(['y', 'y_hat'])
plt.show()

##### Question 2 (10 points)

What is being visualized as y and y_hat on the above plot?

``` Your response here ```

In [ ]:
w,h = si.freqz(b=1,a = a, fs=1)
plt.plot(w,20*np.log10(h))

In [ ]:
z,p,k = si.tf2zpk(B,A)
    
unit_circle = patches.Circle((0,0), radius=1, fill=False, color='black', ls='solid', alpha=0.9)
ax = plt.subplot(111)
ax.add_patch(unit_circle)
ax.spines['left'].set_position('center')
ax.spines['bottom'].set_position('center')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
r = 1.5; plt.axis('scaled'); plt.axis([-r, r, -r, r])
ticks = [-1, -.5, .5, 1]; plt.xticks(ticks); plt.yticks(ticks)    
    
plt.plot(z.real, z.imag, 'ko', fillstyle='none', ms = 10)
plt.plot(p.real, p.imag, 'kx', fillstyle='none', ms = 10)

In [ ]:
D = np.abs(librosa.stft(s,n_fft=256,hop_length = 64))
ld.specshow(librosa.amplitude_to_db(D))

##### Question 3 (20 points)

Record yourself speaking the same vowel sound you analyzed above. 
Graph the power spectral density (PSD) of your recording alongside the PSD of the synthetic signal. 

In [6]:
### Your Code Here


##### Question 4 (10 points)

How does the power spectral density of your recorded signal compare to the LPC spectrum? 

``` Your response here ```

# Simple Singing Vocoder

In this section we will use a spoken sound to process an excitation that plays a melody. In music such an effect is known as vocoding and it is used to produce a talking musical instrument. 

In [1]:
def lpc_to_formants(lpc, sr):    
    """Convert LPC to formants    
    """
        
    # extract roots, get angle and radius
    roots = np.roots(lpc)
    
    pos_roots = roots[np.imag(roots)>=0]
    if len(pos_roots)<len(roots)//2:
        pos_roots = list(pos_roots) + [0] * (len(roots)//2 - len(pos_roots))
    if len(pos_roots)>len(roots)//2:
        pos_roots = pos_roots[:len(roots)//2]
    
    w = np.angle(pos_roots)
    a = np.abs(pos_roots)
    
    order = np.argsort(w)
    w = w[order]
    a = a[order]
    
    freqs = w * (sr/(2*np.pi))
    bws =  -0.5 * (sr/(2*np.pi)) * np.log(a)    
    
    # exclude DC and sr/2 frequencies
    return freqs, bws

##### Questioon 5 [10 points] 

Record yourself speaking slowly the sentence "Mister Blue Sky". Plot a spectrogram of the speech sound.

In [3]:
# Your code here

##### Question 6 [30 points]

In this question we will create a song based on the spoken sentence you recorded. You will choose the melody by creating a sequence of pitches that change slowly over time.
Write a function that does the following:

1. Divide the speech signal into short slices (frames) of 512 samples with 50% overlap
2. For each speech segment compute formants by converting lpc_to_formants (F,Fb)
3. Choose a pitch (f0) for each segment
4. Using the voca function, create a speech sound (vow) from an excitation (ex) with that pitch
4. Overlap and add the sound segments with cross-fade window to create one long sound file 

You are free to alter the durations of the segments and choice of notes for the melody. 
Note that the notes should be relatively long (f0 should not change very often).

Cross-fade between segments can be done by applying a traingular (numpy.bartlett) or raised cosine (numpy.hanning) window to each segment before.

In [ ]:
# Read the speech file
sr, wave = wavfile.read("YOUR_SPEECH_MONO_WAV_FILE.wav")

# precompute the hamming window
frame_len = 512
window = scipy.signal.hann(frame_len)
vocode = np.zeros(len(wave+frame_len))

# 50% window steps for overlap-add
for i in range(0,len(wave),frame_len//2):
    wave_slice = wave[i:i+frame_len]
        
### Your Code Here

    vow,B,A = voca(ex,F,Fb)
    vocode[i:i+frame_len] += vow * window

wavfile.write("YOUR_OUTPUT_FILE.wav", sr, vocode[:len(wave)])

##### Question 7 [10 points]

Why did we use overlapping windows for vocoder? 

``` your response here ```